In [120]:
import torchaudio as ta, torch
from torch import Tensor, nn
import types
from typing import Optional, Tuple
torch.manual_seed(1)

In [121]:
import torchaudio as ta, torch
from torch import Tensor, nn
import types
from typing import Optional, Tuple
torch.manual_seed(1)
model = ta.pipelines.HUBERT_ASR_LARGE.get_model()

# Calculate total number of parameters
total_params = sum(p.numel() for p in model.parameters())

print(f"Total number of parameters in the model: {total_params}")

N1=11000
dummy_data1 = torch.randn([1,N1])

output1 = model(dummy_data1,lengths=torch.tensor([N1]))

N2=22000
dummy_data2 = torch.randn([1,N2])
dummy_data = torch.zeros([2,N2])
dummy_data[0,:N1] = dummy_data1
dummy_data[1,:N2] = dummy_data2

print(dummy_data.shape)

output2 = model(dummy_data,lengths=torch.tensor([N1,N2]))

frames1 = output1[1][0]
print(torch.norm(output1[0][0,:frames1]-output2[0][0,:frames1]))
print(torch.sum(torch.abs(output1[0][0, :frames1])))
print(output1[0][0,:frames1].shape, output2[0][0,:frames1].shape)

Total number of parameters in the model: 94370944
torch.Size([2, 22000])
tensor(68.1878, grad_fn=<LinalgVectorNormBackward0>)
tensor(6021.8594, grad_fn=<SumBackward0>)
torch.Size([34, 768]) torch.Size([34, 768])


In [122]:
output1[0][0,:frames1][:, 0]

tensor([0.4313, 0.4359, 0.4169, 0.4143, 0.4183, 0.4226, 0.4143, 0.3914, 0.4116,
        0.4224, 0.4110, 0.3912, 0.3984, 0.4190, 0.3964, 0.4093, 0.4275, 0.3899,
        0.3975, 0.4138, 0.4407, 0.4229, 0.5912, 0.4292, 0.3778, 0.4014, 0.4262,
        0.4114, 0.4235, 0.5913, 0.4334, 0.3994, 0.3955, 0.4152],
       grad_fn=<SelectBackward0>)

In [123]:
output2[0][0,:frames1][:, 0]

tensor([0.2404, 0.2400, 0.2387, 0.2350, 0.2363, 0.2382, 0.2345, 0.2361, 0.2342,
        0.2362, 0.2337, 0.2351, 0.2377, 0.2384, 0.2376, 0.2328, 0.2360, 0.2354,
        0.2379, 0.2337, 0.2361, 0.2361, 0.2448, 0.2384, 0.2377, 0.2365, 0.2335,
        0.2355, 0.2343, 0.2447, 0.2341, 0.2342, 0.2378, 0.2369],
       grad_fn=<SelectBackward0>)

In [129]:
torch.manual_seed(0)
from speechbrain.lobes.models.huggingface_transformers.hubert import HuBERT
hubert_path = "facebook/hubert-large-ls960-ft"
model_hubert = HuBERT(hubert_path, save_path="/data/LLMs/", freeze=True, freeze_feature_extractor=True).to('cpu')

N1=10000
dummy_data1 = torch.randn([1,N1])

output1 = model_hubert(dummy_data1,wav_lens=torch.tensor([1.0]))

N2=11000
dummy_data2 = torch.randn([1,N2])
dummy_data = torch.zeros([2,N2])
dummy_data[0,:N1] = dummy_data1
dummy_data[1,:N2] = dummy_data2

output2 = model_hubert(dummy_data,wav_lens=torch.tensor([N1/N2,1]))
frames_output1 = output1.shape[1]
print(torch.norm(output1[0, :, :] - output2[0, :frames_output1, :]))
print(torch.sum(torch.abs(output1[0, :, :])))

speechbrain.lobes.models.huggingface_transformers.huggingface - HubertModel is frozen.


tensor(0.5852)
tensor(6839.4707)


In [130]:
output1[0, :, :][:, 0][0:31]

tensor([0.3939, 0.3973, 0.4221, 0.4249, 0.4287, 0.4385, 0.4319, 0.4282, 0.4444,
        0.4253, 0.4181, 0.4258, 0.3998, 0.3166, 0.3974, 0.4423, 0.4318, 0.4283,
        0.4212, 0.4200, 0.4343, 0.4200, 0.4150, 0.4309, 0.4190, 0.3790, 0.4230,
        0.3976, 0.3931, 0.3954, 0.3472])

In [131]:
output2[0, :, :][:, 0][0:31]

tensor([0.3943, 0.3976, 0.4224, 0.4257, 0.4293, 0.4392, 0.4338, 0.4291, 0.4454,
        0.4268, 0.4190, 0.4268, 0.4010, 0.3221, 0.3974, 0.4432, 0.4330, 0.4294,
        0.4214, 0.4209, 0.4351, 0.4209, 0.4157, 0.4315, 0.4201, 0.3688, 0.4240,
        0.3986, 0.3941, 0.3960, 0.3472])

In [24]:
for name, layer in model_hubert.named_modules():
    if isinstance(layer, torch.nn.GroupNorm):
        print(f"GroupNorm found: {name}")
    elif isinstance(layer, torch.nn.LayerNorm):
        #$print(f"LayerNorm found: {name}")
        pass
    elif isinstance(layer, nn.BatchNorm1d) or isinstance(layer, nn.BatchNorm2d) or isinstance(layer, nn.BatchNorm3d):
        print(f"BatchNorm found: {name}")